#Getting Started
This notebook demonstrates a fundamental multi-agent workflow built using LangGraph. We will construct a graph with a supervisor node that dynamically routes tasks to specialized agent nodes: a researcher and a calculator. The core of this workflow is managed by a LangGraph StateGraph, where the state evolves as the agents interact and perform their designated roles.


### Install Required Packages

In [ ]:
%pip install --upgrade --user --quiet "google-cloud-aiplatform[evaluation, langchain, reasoningengine]" \
    "langchain_google_vertexai" \
    "langgraph" \
    "cloudpickle==3.0.0" \
    "pydantic>=2.10" \
    "requests"

### Restart current runtime

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

### Authenticate your notebook environment (Colab only)

In [ ]:
from google.colab import auth
auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI

In [ ]:
# Use the environment variable if the user doesn't provide Project ID.
import os
import vertexai

PROJECT_ID = ""  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
LOCATION ="us-central1"

vertexai.init(project=PROJECT_ID,location=LOCATION,)

### Import libraries

In [ ]:
from langchain_core.tools import tool
from langgraph.prebuilt import tools_condition
from langgraph.prebuilt import ToolNode
from IPython.display import Image, display
from langchain_google_vertexai import ChatVertexAI
from langgraph.prebuilt import create_react_agent
from typing import TypedDict, List, Tuple, Dict, Any
from typing import Literal
from langgraph.graph import MessagesState,StateGraph,END,START
from langchain_core.messages import HumanMessage
from langgraph.types import Command
from vertexai.generative_models import (GenerationConfig,GenerativeModel,Tool, grounding)

### Define Search Tool

In [ ]:
@tool
def search_tool(prompt: str)->str:
    """
    Use Google Search for grounding.

    This function utilizes the Google Search tool to retrieve information relevant to the given prompt.
    It employs dynamic retrieval to adjust the search threshold based on relevance.
    The function generates content using the provided prompt and tools, and returns the response.
    """
    model = GenerativeModel("gemini-1.5-pro")
    tool = Tool.from_google_search_retrieval(
        grounding.GoogleSearchRetrieval(
            # Optional: For Dynamic Retrieval
            dynamic_retrieval_config=grounding.DynamicRetrievalConfig(
                mode=grounding.DynamicRetrievalConfig.Mode.MODE_DYNAMIC,
                dynamic_threshold=0.7,
            )
        )
    )
    response = model.generate_content(
        prompt,
        tools=[tool],
        generation_config=GenerationConfig(
            temperature=0.0,
        ),
    )

    return(response.text)


In [ ]:
#Test the tool
search_tool("Which is the capital city of India ?")

### Define model

In [ ]:
llm=ChatVertexAI(model_name="gemini-2.0-flash-001")

## Define Research Agent

In [ ]:
# The 'create_react_agent' function in LangGraph's pre-built templates provides
# a streamlined way to create a ReAct agent. ReAct (Reasoning and Acting) is a popular framework
# for building agents that can interact with tools.
# This function encapsulates the core components and logic needed to set up such an agent.
research_agent = create_react_agent(llm, tools=[search_tool], prompt="You are a researcher. DO NOT do any math.")

### Define Python functions (tools)

In [ ]:
def multiply(a: int, b: int) -> int:
    """Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b

# This will be a tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b

def divide(a: int, b: int) -> float:
    """Divide a and b.

    Args:
        a: first int
        b: second int
    """
    return a / b

my_tools = [add, multiply, divide]




## Define Math Agent

In [ ]:
math_agent = create_react_agent(llm, tools=my_tools)

## Start Building the Graph

In [ ]:
members = ["researcher", "calculator"]
# Our team supervisor is an LLM node. It just picks the next agent to process
# and decides when the work is completed
options = members + ["FINISH"]

system_prompt = (
    "You are a supervisor tasked with managing a conversation between the"
    f" following workers: {members}. Given the following user request,"
    "Respond FINISH if answer is correct"
    "Respond with the worker to act next. "

)

class Router(TypedDict):
    """Worker to route to next. Once task is done, route to FINISH."""
    next: Literal["researcher", "calculator", "FINISH"]


class State(MessagesState):
    next: str

## Define Supervisor Node

In [ ]:
def supervisor_node(state: State) -> Command[Literal["researcher", "calculator", "__end__"]]:
    messages = [
        {"role": "system", "content": system_prompt},
    ] + state["messages"]
    response = llm.with_structured_output(Router).invoke(messages)
    print("\n---This is what superviser see:")
    print(response)
    print("\n-----")
    goto = response["next"]
    if goto == "FINISH":
        goto = "__end__"

    if (len(state["messages"]) >= 2):
      goto = "__end__"

    return Command(goto=goto, update={"next": goto})


## Define Research Node

In [ ]:
def research_node(state: State) -> Command[Literal["supervisor"]]:
    result = research_agent.invoke(state)
    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="researcher")
            ]
        },
        goto="supervisor",
    )

Define Calculator Node

In [ ]:
def calculator_node(state: State) -> Command[Literal["supervisor"]]:
    result = math_agent.invoke(state)
    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="calculator")
            ]
        },
        goto="supervisor",
    )

## Define Grpah Structure

In [ ]:
builder = StateGraph(State)
builder.add_edge(START, "supervisor")
builder.add_node("supervisor", supervisor_node)
builder.add_node("researcher", research_node)
builder.add_node("calculator",calculator_node)
graph = builder.compile()

In [ ]:
from IPython.display import display, Image
display(Image(graph.get_graph().draw_mermaid_png()))

# Execute the query

In [ ]:
for s in graph.stream(
    {"messages": [("user", "What is capital city of India? ")]}, subgraphs=True):
    print(s)
    print("----")